# TP Jointure

In [2]:
import pandas as pd
import json, copy, pymongo
# code moi steupléé 🥺

# approche par réservation (simple)
reservations = pd.read_csv("bddPilotes/RESERVATIONS.txt", sep="\t", header=0).to_dict('records')
vols = pd.read_csv("bddPilotes/VOLS.txt", sep="\t", header=0).to_dict('records')
pilotes = pd.read_csv("bddPilotes/PILOTES.txt", sep="\t", header=0).to_dict('records')
clients = pd.read_csv("bddPilotes/CLIENTS.txt", sep="\t", header=0).to_dict('records')
avions = pd.read_csv("bddPilotes/AVIONS.txt", sep="\t", header=0).to_dict('records')

# approche pour connaitre les vols (dénormalisation des données)
# ceci est un exemple car nous faisons donc une démultiplication des informations. Ici nous partons du principe qu'une personne arrivant dans l'aéroport veut connaître ces informations, et voir son nom associé au vol. Nous croisons donc les informations, ce qui limitera les besoins de jointure à la volée par la suite.
vue_vols = copy.deepcopy(vols)
for v in vue_vols:
    for a in avions:
        if a['NUMAV'] == v["NUMAV"]:
            v.update({"avion": {"NOMAV": a["NOMAV"], "LOCALISATION": a["LOCALISATION"]}})
    v_clients = []
    for r in reservations:
        if r["VOLID"] == v["VOLID"]: 
            v_clients.append(r["NUMCLIENT"])
    clients_names = []
    for c in clients:
        if c["CLIENTID"] in v_clients: 
            clients_names.append(c["NOM"])
    v.update({"clients": clients_names})
    del v["NUMPIL"]

print(vue_vols[0])

# une autre approche est ella par pilote : un pilote veut savoir son heure de départ, son nom associé à un avion et son modèle d'avion avec capacité et localisation. Il n'a pas besoin du reste.
vue_pilotes = copy.deepcopy(vols)

for vp in vue_pilotes:
    for a in avions:
        if a['NUMAV'] == vp["NUMAV"]:
            vp.update({"avion": a})
            del vp["NUMAV"]
            break
    for p in pilotes:
        if p["NUMPIL"] == vp["NUMPIL"]:
            vp.update({"pilote": p})
            del vp["NUMPIL"]
            break

vue_pilotes[0]

{'VOLID': 'V101', 'VILLEDEPART': 'Marseille', 'VILLEARRIVEE': 'Amsterdam', 'DATEDEPART': '1/04/07', 'HEUREDEPART': '8:10', 'DATEARRIVEE': '1/04/07', 'HEUREARRIVEE': '9:10', 'NUMAV': 560, 'avion': {'NOMAV': 'Airbus A320', 'LOCALISATION': 'Nice'}, 'clients': ['Leblanc', 'Grignard', 'Rolland', 'Perse', 'Lorentz', 'Lenard', 'Planck', 'Bohr', 'Dirac']}


{'VOLID': 'V101',
 'VILLEDEPART': 'Marseille',
 'VILLEARRIVEE': 'Amsterdam',
 'DATEDEPART': '1/04/07',
 'HEUREDEPART': '8:10',
 'DATEARRIVEE': '1/04/07',
 'HEUREARRIVEE': '9:10',
 'avion': {'NUMAV': 560,
  'NOMAV': 'Airbus A320',
  'CAPACITE': 250,
  'LOCALISATION': 'Nice'},
 'pilote': {'NUMPIL': 5050,
  'NOMPIL': 'Leblanc',
  'NAISSANCE': 1960,
  'ADRESSE': 'Marseille'}}